In [ ]:
from typing import Literal, Callable, Any
import math, time, json, random
from random import randint
from collections import defaultdict

from drivingppo.common import set_seed, SPD_SCFAC, WORLD_DT, ACTION_REPEAT, LOOKAHEAD_POINTS
from drivingppo.environment import speed_norm, get_pp_lookahead_point
from drivingppo.world import World, Car, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg

from world_samples import NEAR, gen_0, gen_1, gen_2, gen_2l, generate_random_world_plain, gen_obs

import numpy as np
import pandas as pd

W_CONFIG = {
    'map_border': False,
    'near': NEAR,
    'far': 999.9,
}


class PurePursuitController:
    def __init__(self,
                 target_speed=12.0,
                 min_ld=1.5,
                 ld_k=1.0,
                 kp_steer=1.5,
                 k_cornering=0.2,
                 k_accel=0.7,
    ):
        self.target_speed = target_speed # 목표 주행 속도 (m/s)
        self.min_ld       = min_ld       # 전방 주시 거리 최소 (m)
        self.ld_k         = ld_k         # 속도 비례 전방 주시 계수 (Ld = min_ld + ld_k * speed)
        self.kp_steer     = kp_steer     # 조향 비례 제어 게인
        self.k_cornering  = k_cornering  # 선회시 감속 정도
        self.k_accel      = k_accel      # 종방향 가속 게인

    def get_action(self, world:World) -> tuple[float, float, bool]:
        """
        World 개체를 받아 액션(ws, ad, stop) 반환
        """
        p = world.player
        px, pz = p.x, p.z

        # 이미 도착한 경우 정지
        if world.arrived:
            return 0.0, 0.0, True

        ld = self.min_ld + self.ld_k * max(0, p.speed)
        lookahead_point, _ = get_pp_lookahead_point(world, ld)

        # 조향 제어 (ad)
        tx, tz = lookahead_point
        abs_angle = angle_of(px, pz, tx, tz)

        rel_angle = abs_angle - p.angle_x
        rel_angle = (rel_angle + math.pi) % (2*math.pi) - math.pi  # 정규화: -pi ~ pi

        ad = self.kp_steer * rel_angle
        # 종방향 제어 (ws)
        current_target_speed = self.target_speed * (1.0 - ad*ad * self.k_cornering)
        ad = max(min(ad, 1.0), -1.0)  # 액션 유효범위로 클리핑

        speed_error = current_target_speed - p.speed
        ws = speed_error * 0.5  # 가속도 비례 게인
        ws = max(min(ws, 1.0), -1.0)

        return ws, ad, False


def evaluate(
        con:PurePursuitController,
        world_generator: Callable[[], World],
        episode_num=100,
        verbose: bool=True,
        csv_path: str="",
        time_step=WORLD_DT,
        action_repeat=ACTION_REPEAT,
        max_esteps=2000  # 무한 루프 방지용 최대 스텝 수
) -> dict:

    print(f"Pure Pursuit {episode_num}회 에피소드 평가 시작...")

    all_metrics = defaultdict(list)
    episode_lengths = []
    success_count = 0

    for i in range(episode_num):
        i1 = i + 1
        checkPeriod = max(1, episode_num // 10)

        world = world_generator()

        estep_count = 0
        ws_penalty_sum = 0.0
        ad_penalty_sum = 0.0
        speed_list = []
        action_history = []

        done = False

        # 에피소드 진행
        while not done and estep_count < max_esteps:
            estep_count += 1

            ws, ad, _ = con.get_action(world)
            action_history.append([ws, ad])

            # --- 물리 스텝 실행 ---
            for _ in range(action_repeat):
                world.set_action(ws, ad)
                world.step(time_step)

            # --- 지표 수집 ---
            s_norm = speed_norm(world.player.speed)
            ws_penalty = (- ws * s_norm * 4.0) if (ws * s_norm > 0) else 0.0

            ws_penalty_sum += ws_penalty
            ad_penalty_sum += (- ad * ad * 1.7)
            speed_list.append(world.player.speed)

            # --- 종료 조건 검사 (도달, 경로 이탈, 충돌 등) ---
            if world.arrived or getattr(world, 'lost', False) or getattr(world, 'player_collision', False):
                done = True

        # 에피소드 단위 지표 집계
        tcount = estep_count * action_repeat * time_step / 1000.0
        final_ws = ws_penalty_sum / tcount if tcount > 0 else 0.0
        final_ad = ad_penalty_sum / tcount if tcount > 0 else 0.0
        speed_mean = np.mean(speed_list) if speed_list else 0.0
        speed_var  = np.var(speed_list) if speed_list else 0.0
        if len(action_history) > 0:
            action_arr = np.array(action_history)
            ws_diff_mean = float(np.mean(np.abs(np.diff(action_arr[:, 0]))))  if len(action_history) > 1  else 0.0
            ad_sq_mean   = float(np.mean(np.square(action_arr[:, 1])))
        else:
            ws_diff_mean, ad_sq_mean = 0.0, 0.0

        is_success = world.arrived
        if is_success: success_count += 1

        episode_lengths.append(estep_count)

        all_metrics['ending/estep'].append(estep_count)
        all_metrics['current_time'].append(tcount)
        all_metrics['rewards/6.ws'].append(final_ws)
        all_metrics['rewards/7.ad'].append(final_ad)
        all_metrics['metrics/speed_mean'].append(speed_mean)
        all_metrics['metrics/speed_var'].append(speed_var)
        all_metrics['metrics/ws_diff_mean'].append(ws_diff_mean)
        all_metrics['metrics/ad_sq_mean'].append(ad_sq_mean)
        all_metrics['success'].append("True" if is_success else "False") # 범주형 출력을 위해 문자열 처리

        # 진행 상황 출력
        if i1 % checkPeriod == 0:
            if verbose:
                status = "성공" if is_success else "실패/이탈"
                print(f"[{i1}/{episode_num}] 완료 - 상태: {status}, Steps: {estep_count}, Avg Speed: {speed_mean:.2f}")

    # 평가 결과 출력 및 저장
    if verbose:
        print("\n" + "="*41)
        print(f"평가 결과 ({episode_num} 에피소드 평균)")
        print("="*41)
        print(f"Success Rate  : {(success_count / episode_num) * 100:.1f}% ({success_count}/{episode_num})")
        print(f"Episode Length: {np.mean(episode_lengths):.1f} ± {np.std(episode_lengths):.1f}")

    if all_metrics:
        df_metrics = pd.DataFrame(all_metrics)

        # 수치형: 평균, 표준편차
        num_df = df_metrics.select_dtypes(include=[np.number])
        if not num_df.empty and verbose:
            print("-" * 41)
            summary = num_df.describe().loc[['mean', 'std']].T
            summary['mean'] = summary['mean'].map('{:.4f}'.format)  # 지수표기 안 함.
            print(summary)

        # 문자열(범주형): 종류별 비율
        cat_df = df_metrics.select_dtypes(exclude=[np.number])
        if not cat_df.empty and verbose:
            print("-" * 41)
            for col in cat_df.columns:
                print(f"\n* {col}")
                counts = df_metrics[col].value_counts()
                ratios = df_metrics[col].value_counts(normalize=True)

                for idx, val in counts.items():
                    ratio = ratios[idx] * 100  #type:ignore
                    print(f"   - {idx:<10}: {val:3d}회 ({ratio:5.1f}%)")

        # CSV 저장
        if csv_path:
            df_metrics.to_csv(csv_path, index=False)
            if verbose:
                print(f"\n💾 세부 결과 저장됨: {csv_path}")

    return all_metrics

In [ ]:
from drivingppo.simsim import WorldViewer

def testew():
    obstacle_map = create_empty_map(100, 100)
    w = World(
        wh=(100, 100),
        player=Car({
            'playerPos': {'x': 10, 'z': 10},
            'playerBodyX': 0.0,
            'playerSpeed': 0.,
        }),
        obstacle_map=obstacle_map,
        waypoints=[
            (10, 50),
            (20, 50),
            (50, 70),
            (60, 60),
        ],
        config=W_CONFIG|{
            'map_border': False,
            'far': 999.9
        }
    )
    return w


def run(
        world_generator:Callable[[], World],
        con:PurePursuitController,
        time_spd=2.0,
        time_step=WORLD_DT,
        action_repeat=ACTION_REPEAT,
        auto_close_at_end=True,
    ):

    world = world_generator()
    viewer = WorldViewer(world, auto_update=False)

    # 지표 수집용 변수 초기화
    estep_count = 0
    ws_penalty_sum = 0.0
    ad_penalty_sum = 0.0
    speed_list = []

    while not world.arrived:

        estep_count += 1
        ws, ad, _ = con.get_action(world)  # 컨트롤러가 조작 산출

        s_norm = speed_norm(world.player.speed)
        ws_penalty = (- ws * s_norm * 4.0) if (ws * s_norm > 0) else 0.0
        ws_penalty_sum += ws_penalty
        ad_penalty_sum += (- ad * ad * 1.7)
        speed_list.append(world.player.speed)

        for _ in range(action_repeat):
            world.set_action(ws, ad)  # 행동 실행
            world.step(time_step)

        if viewer.closed: break
        viewer.update()  # 시각화 호출
        time.sleep(time_step / 1000.0 / time_spd)  # 시각화 프레임을 위해 딜레이 추가

    tcount = estep_count * action_repeat * time_step / 1000.0  # 흐른 시간 (초)
    final_ws = ws_penalty_sum / tcount
    final_ad = ad_penalty_sum / tcount
    speed_mean = np.mean(speed_list)  if speed_list else 0.0
    speed_var  = np.var(speed_list)   if speed_list else 0.0

    print("\n=== Pure Pursuit 에피소드 종료 지표 ===")
    print(f'current_time: {tcount}')
    print(f'ending/estep: {estep_count}')
    print(f"rewards/6.ws      : {final_ws:.4f}")
    print(f"rewards/7.ad      : {final_ad:.4f}")
    print(f"metrics/speed_mean: {speed_mean:.4f}")
    print(f"metrics/speed_var : {speed_var:.4f}")
    print("=======================================\n")

    if auto_close_at_end:
        viewer.close()
    else:
        viewer.occupy_mainloop()

def gen_2l(): return generate_random_world_plain(map_h=150, map_w=150, num=10,               min_dist=15,  max_dist=30, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')

con = PurePursuitController()
set_seed(7)
run(gen_2l, con, auto_close_at_end=False)

In [ ]:
def gen_2l(): return generate_random_world_plain(map_h=150, map_w=150, num=10,               min_dist=10,  max_dist=30, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')

set_seed(20)
metrics = evaluate(
    con=PurePursuitController(),
    world_generator=gen_2l,
    episode_num=100,
    verbose=True,
    csv_path="pp.csv"
)

In [ ]:
# 최적 파라미터 찾기
from typing import Callable
import os
from collections import defaultdict
import itertools

from drivingppo.common import set_seed, WORLD_DT, ACTION_REPEAT
from drivingppo.environment import speed_norm
from drivingppo.world import World
from world_samples import NEAR, gen_0, gen_1, gen_2, gen_2l, generate_random_world_plain

import numpy as np
import pandas as pd

FILE_PATH = "pp_grid_search_results.csv"

# 파라미터 탐색 공간
param_grid = {
    'target_speed': [10.0, 12.0, 14.0],
    'min_ld':      [1.3, 1.4, 1.5, 1.6, 1.7],
    'ld_k':        [0.5, 0.8, 1.0, 1.2],
    'kp_steer':    [1.3, 1.5, 1.7],
    'k_cornering': [0.0, 0.1, 0.2, 0.3],
    'k_accel':     [0.4, 0.6, 0.8, 1.0],
}


def evaluate_pp_fast(
        con:PurePursuitController,
        world_generator: Callable[[], World],
        episode_num=100,
        # verbose:bool=False,
        max_esteps=230,         # 1차 컷오프: 단일 에피소드 스텝 상한 (너무 느리면 탈락)
        early_stop_mean=138.0,  # 2차 컷오프: RL 모델을 이길 가능성이 없는 평균 한계선
        min_check_eps=15        # 2차 컷오프 발동을 위한 최소 평가 횟수
) -> dict | None:

    all_metrics = defaultdict(list)
    episode_lengths = []

    for i in range(episode_num):
        world = world_generator()

        estep_count = 0
        ws_penalty_sum = 0.0
        ad_penalty_sum = 0.0
        speed_list = []
        action_history = []

        done = False

        # 물리 시뮬레이션 진행
        while not done and estep_count < max_esteps:
            estep_count += 1

            ws, ad, _ = con.get_action(world)
            action_history.append((ws, ad))

            # 지표 수집
            s_norm = speed_norm(world.player.speed)
            ws_penalty = (- ws * s_norm * 4.0) if (ws * s_norm > 0) else 0.0

            ws_penalty_sum += ws_penalty
            ad_penalty_sum += (- ad * ad * 1.7)
            speed_list.append(world.player.speed)

            # 행동 적용
            for _ in range(ACTION_REPEAT):
                world.set_action(ws, ad)
                world.step(WORLD_DT)

            # 종료 조건 검사
            if world.arrived or getattr(world, 'lost', False) or getattr(world, 'player_collision', False):
                done = True

        # 1차 조기 종료: 최대 소요스텝수 제한
        if not world.arrived or estep_count >= max_esteps:
            return None

        # 에피소드 지표 집계
        tcount = estep_count * ACTION_REPEAT * WORLD_DT / 1000.0
        speed_mean = np.mean(speed_list) if speed_list else 0.0
        speed_var  = np.var(speed_list) if speed_list else 0.0
        if len(action_history) > 0:
            action_arr = np.array(action_history)
            ws_diff_mean = float(np.mean(np.abs(np.diff(action_arr[:, 0]))))  if len(action_history) > 1  else 0.0
            ad_sq_mean   = float(np.mean(np.square(action_arr[:, 1])))
        else:
            ws_diff_mean, ad_sq_mean = 0.0, 0.0

        episode_lengths.append(estep_count)

        all_metrics['ending/estep'].append(estep_count)
        all_metrics['current_time'].append(tcount)
        all_metrics['metrics/ws_diff_mean'].append(ws_diff_mean)
        all_metrics['metrics/ad_sq_mean'].append(ad_sq_mean)
        all_metrics['metrics/speed_mean'].append(speed_mean)
        all_metrics['metrics/speed_var'].append(speed_var)
        all_metrics['success'].append("True") 

        # 2차 조기 종료 (Soft Cut-off)
        # 근거: 초반 운이 좋아 1차 컷오프를 통과했더라도, 00번 이상 누적된 평균이 000.0을 초과하면 가망이 없음
        if (i + 1) >= min_check_eps:
            current_mean = sum(episode_lengths) / len(episode_lengths)
            if current_mean > early_stop_mean:
                return None 

    # 모든 검증을 무사히 통과한 우수 조합만 데이터 반환
    return all_metrics


def run_grid_search_itertools(world_generator: Callable[[], World], output_csv:str=FILE_PATH, delete_prev=True, seed=0):
    if delete_prev and os.path.exists(output_csv):
        os.remove(output_csv)

    # 파라미터 조합 생성
    keys   = param_grid.keys()
    values = param_grid.values()
    combinations = list(itertools.product(*values))

    total_combinations = len(combinations)
    print(f"파라미터 총 {total_combinations}개 조합 탐색 시작")

    for current_iter, combo in enumerate(combinations, 1):
        params = dict(zip(keys, combo))
        print(f"[{current_iter}/{total_combinations}] 평가 중: {params}", end="")

        con = PurePursuitController(**params)
        set_seed(seed)

        # 조기 종료가 적용된 빠른 평가 함수 호출
        metrics = evaluate_pp_fast(con, world_generator, episode_num=100)

        # 조기 종료 발동 시 (None 반환) CSV 기록 생략하고 다음으로 진행
        if metrics is None:
            print(" -> ❌ 성능 미달 (조기 종료)")
            continue

        print(" -> ✅ 성공 (기록됨)")

        # 결과 집계 및 병합
        current_results = []
        num_episodes = len(metrics.get('ending/estep', []))
        for i in range(num_episodes):
            row_data = params.copy()
            for key in metrics.keys():
                row_data[key] = metrics[key][i]
            current_results.append(row_data)

        df_iter = pd.DataFrame(current_results)

        write_header = not os.path.exists(output_csv)  # 파일이 없을 때만 헤더 작성
        df_iter.to_csv(output_csv, mode='a', header=write_header, index=False)

    print(f"\n탐색 완료. 생존한 최적 데이터 저장: '{output_csv}'")


def analyze_grid_search_results(file_path=FILE_PATH, verbose=True):
    df = pd.read_csv(file_path)

    # 범주형(True,False) --> 수치(1,0) 평균 계산하게
    if 'success' in df.columns and df['success'].dtype == object:
        df['success'] = df['success'].map({'True': 1.0, 'False': 0.0})

    param_cols = ['min_ld', 'ld_k', 'kp_steer', 'k_cornering']  # 그룹기준: pp 파라미터 조합

    # 그룹화, 모든 수치형 지표의 평균
    grouped_df = df.groupby(param_cols).mean().reset_index()

    # 정렬
    sorted_df = grouped_df.sort_values(
        by=['success', 'ending/estep'],
        ascending=[False, True]
    )

    if verbose:
        print(sorted_df.to_string(index=True))

    return sorted_df



# run_grid_search_itertools(gen_2l)
summary = analyze_grid_search_results(verbose=False)
summary.head(10)